In [1]:
import model_chains_config
import ast
import pickle
import pandas as pd
import numpy as np

In [2]:
# import warnings
# warnings.filterwarnings('ignore')
pd.set_option('display.float_format',lambda x: '%.2f' % x)

In [3]:
def col_types(data):
    column_types = data.dtypes.to_frame().reset_index()
    column_types.columns = ['column_name', 'type']
    column_types.sort_values( by='type', inplace = True)
    return column_types

In [ ]:
with open('../Data/meta/test_x_after_feature_engineering.pickle', 'rb') as f:
    data_x = pickle.load(f)
    
data_x

In [51]:
col_types(data_x)

,ColumnName,Type
0,clean_pressure31,float64
29,painting_g7_act_hvv,float64
30,painting_g9_act_hvv,float64
32,painting_g11_act_hvv,float64
33,painting_g12_act_a_air,float64
34,env_rpi05_hum,float64
35,env_rpi05_pm1,float64
36,env_rpi07_hum,float64
37,env_rpi07_pm10,float64
38,env_rpi07_temp,float64


In [52]:
# onehot encoding
for index , col in enumerate(data_x.columns.tolist()):
    if data_x[col].dtype == 'object':
        data_x = data_x.join(pd.get_dummies(data_x[col],prefix=col))
        data_x = data_x.drop(col, axis = 1)

In [53]:
# onehot encoding後有些train的欄位在測試集沒有，補上欄位

for i in model_chains_config.x_train_columns:
    if i not in data_x.columns.tolist():
        data_x[i] = np.zeros(100, dtype='uint8')

# 欄位排序
data_x = data_x[model_chains_config.x_train_columns]

In [ ]:
col_types(data_x)

In [57]:
X_PT = data_x.copy()
predict_y_test = pd.DataFrame({})

for k in range(6):
    
    with open('../model/model_Y'+ str(model_chains_config.order_list[k]) + '(Albert).pickle', 'rb') as f:
        model = pickle.load(f)
    
    # 儲存預測結果
    predict_y_test[model_chains_config.data_y_col[model_chains_config.order_list[k]]] = model.predict(X_PT)
    # 把取得的預測值當作變數放進X
    X_PT[model_chains_config.data_y_col[model_chains_config.order_list[k]]] = model.predict(X_PT)
    
    # 清空 model
    del model
    
# 調整欄位順序
predict_y_test = predict_y_test[model_chains_config.data_y_col]
display(predict_y_test)

with open('../Output/predict_y_test(Albert).pickle', 'wb') as f:
    pickle.dump(predict_y_test, f)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-1.42,-0.72,-0.91,-1.27,-1.04,-0.98
1,-1.24,-0.57,-1.02,-1.73,-0.61,-0.81
2,-1.27,-0.58,-1.02,-1.73,-0.61,-0.82
3,-0.04,-1.08,-1.24,-1.53,-1.03,-0.60
4,-0.04,-1.08,-1.24,-1.53,-1.03,-0.60
5,-0.21,-1.08,-1.14,-1.51,-0.98,-0.51
6,-0.48,-1.02,-1.24,-1.53,-1.04,-0.65
7,0.38,-0.25,-0.73,-0.48,-0.30,-0.08
8,0.27,-0.42,0.14,0.11,-0.48,-0.38
9,0.14,-0.42,0.14,0.11,-0.48,-0.41
